# Project 3: Digital Public Services and Economic Development

Governments around the world are increasingly investing in digital public services to reduce administrative burdens, improve transparency, and support private sector activity. Online business registration, electronic tax filing, and digital licensing systems are often framed as tools that can improve the business environment and promote economic growth.

The World Bank’s Business Ready (B-READY) project provides a standardized way to measure how effectively economies design regulations, deliver public services, and implement rules in practice. One of its three pillars, **Public Services**, captures the availability and quality of business-facing public services, many of which are delivered digitally.

This project explores whether stronger performance in public services is associated with higher levels of economic development. Rather than testing causality, the analysis uses cross-country data to examine patterns and variation across economies.



## What is the B-READY Project?

The Business Ready (B-READY) project is the World Bank’s flagship framework for assessing the quality of the business environment across economies. It replaces the earlier Doing Business indicators and places greater emphasis on how regulations and public services function in practice.

B-READY is structured around three pillars:
1. Regulatory Framework
2. Public Services
3. Operational Efficiency

Each pillar is composed of multiple **topics**, such as Business Entry, Utility Services, Taxation, and Dispute Resolution. For each topic, economies receive scores based on standardized questionnaires and expert assessments.

Importantly, **Public Services is not provided as a single standalone variable**. Instead, it is a pillar composed of topic-level scores. Analysts are expected to aggregate these topic scores to construct a pillar-level measure.



## Data Sources

This project uses only official World Bank data:

1. **Business Ready (B-READY) 2024 – Pillar Scores**
   - Economy-level scores for:
     - Public Services
     - Regulatory Framework
     - Operational Efficiency
   - Coverage: 50 economies

2. **World Development Indicators (WDI)**
   - GDP per capita (current US dollars)
   - Used as a broad measure of economic development

The B-READY data are drawn from the World Bank’s Business Ready 2024 release, which includes updated scoring corrections and methodological documentation.

## Research Question

Is higher income per capita associated with stronger performance in business-facing public services, as measured by the B-READY Public Services pillar?

## Hypothesis

Economies with higher GDP per capita tend to score higher on B-READY Public Services. However, substantial variation is expected, suggesting that strong public services can also emerge in lower-income contexts.

And with this, let's start importing our libraries!

# Step 0 – Importing Libraries

In [ ]:
import plotly.io as pio

pio.renderers.default = "notebook_connected+plotly_mimetype"

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


# Step 1 – Loading the Data

## Loading the B-READY Data

We begin by loading the B-READY Excel file that contains economy-level scores by pillar and topic. This file includes observations for each economy, broken down by pillar and topic.

The first step is to inspect the structure of the dataset to understand how pillars, topics, and scores are represented.


In [41]:
# Load Data
import pandas as pd

bready_df = pd.read_csv(
    "01_B-READY-2024-PILLAR-TOPIC-SCORES-2024.csv", sep=",", decimal="."
)


bready_df.head(20)

# Clean column names
bready_df.columns = bready_df.columns.str.replace("\n", " ", regex=False).str.strip()


# Rename columns to clean, python-friendly names
df = df.rename(
    columns={
        "Pillar 1 Regulatory Framework": "regulatory_framework",
        "Pillar 2 Public Services": "public_services",
        "Pillar 3 Operational Efficiency": "operational_efficiency",
        "gdp_per_capita_2024": "gdp_per_capita",
    }
)

df.columns


bready_df.head(20)


,Economy,Economy Code,Pillar 1 Regulatory Framework,Pillar 2 Public Services,Pillar 3 Operational Efficiency,Unnamed: 5
0,Bangladesh,BGD,56.94,41.64,70.49,NaN
1,Barbados,BRB,58.77,46.40,66.55,NaN
2,Bosnia and Herzegovina,BIH,67.44,37.81,70.05,NaN
3,Botswana,BWA,65.96,48.52,67.73,NaN
4,Bulgaria,BGR,76.33,64.03,74.82,NaN
5,Cambodia,KHM,62.83,39.14,60.66,NaN
6,Central African Republic,CAF,60.99,18.35,40.36,NaN
7,Chad,TCD,61.15,23.51,48.05,NaN
8,Colombia,COL,76.60,66.28,66.38,NaN
9,Costa Rica,CRI,71.34,63.58,53.66,NaN


## Loading GDP per Capita Data

GDP per capita data are drawn from the World Bank’s World Development Indicators. The most recent available year is used to provide a snapshot of economic development across economies.



In [ ]:
gdp = pd.read_csv("API_NY.GDP.PCAP.CD_DS2_en_csv_v2_2.csv", skiprows=4)
gdp.head()


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,27441.550214,28440.041688,30082.158423,30645.890602,22759.807175,26749.329609,30975.998912,35718.753119,39498.594129,NaN
1,Africa Eastern and Southern,AFE,GDP per capita (current US$),NY.GDP.PCAP.CD,186.089204,186.909053,197.367547,225.400079,208.962717,226.836135,...,1334.470500,1528.104224,1552.073722,1507.085600,1351.591669,1562.416175,1679.327622,1571.449189,1615.396356,NaN
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,522.082216,525.469771,491.337221,496.602504,510.787063,356.496214,357.261153,413.757895,NaN,NaN
3,Africa Western and Central,AFW,GDP per capita (current US$),NY.GDP.PCAP.CD,121.936832,127.451040,133.823783,139.004980,148.545883,155.561897,...,1630.039439,1574.230564,1720.140092,2216.385055,2030.861659,2112.794076,2138.473153,1841.855064,1411.337029,NaN
4,Angola,AGO,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2051.814621,2790.718869,2860.093648,2493.678844,1759.356199,2303.908127,3682.113151,2916.136633,2665.874448,NaN


The World Development Indicators dataset includes GDP per capita values for many years.  
For this analysis, only the most recent year (2024) is retained to provide a current snapshot of economic development.

In [42]:
# Keep only country code and 2024 GDP per capita
gdp_2024 = gdp[["Country Code", "2024"]].rename(
    columns={"Country Code": "Economy Code", "2024": "gdp_per_capita_2024"}
)

# Drop rows where GDP data is missing
gdp_2024 = gdp_2024.dropna()

gdp_2024.head()


,Economy Code,gdp_per_capita_2024
0,ABW,39498.594129
1,AFE,1615.396356
3,AFW,1411.337029
4,AGO,2665.874448
5,ALB,11377.775743


The B-READY and GDP datasets are merged using economy (ISO) codes. This creates a single dataset that links public service performance with income levels.


In [ ]:
df = pd.merge(bready_df, gdp_2024, on="Economy Code", how="inner")

df.head()


,Economy,Economy Code,Pillar 1 Regulatory Framework,Pillar 2 Public Services,Pillar 3 Operational Efficiency,Unnamed: 5,gdp_per_capita_2024
0,Bangladesh,BGD,56.94,41.64,70.49,NaN,2593.416117
1,Barbados,BRB,58.77,46.40,66.55,NaN,26544.870728
2,Bosnia and Herzegovina,BIH,67.44,37.81,70.05,NaN,9358.787690
3,Botswana,BWA,65.96,48.52,67.73,NaN,7695.753194
4,Bulgaria,BGR,76.33,64.03,74.82,NaN,17596.017366


## Scatter Plot: GDP per Capita vs Public Services

This scatterplot visualizes whether higher income per capita is associated with stronger performance in business-facing public services (B-READY Public Services).

Each point represents one economy.  
Because GDP per capita is typically very skewed (some countries are extremely high-income), we use a log-scale on the x-axis to make the pattern easier to see.


In [37]:
import plotly.express as px

fig = px.scatter(
    df,
    x="gdp_per_capita",
    y="public_services",
    hover_name="Economy",
    title="GDP per Capita and Business-Facing Public Services (B-READY)",
    labels={
        "gdp_per_capita": "GDP per Capita (current US$)",
        "public_services": "B-READY Public Services Score",
    },
    log_x=True,
)


fig.update_traces(marker=dict(size=10, opacity=0.75), selector=dict(mode="markers"))

fig.update_layout(
    template="plotly_white",
    title_x=0.5,
    font=dict(size=14),
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
)

fig.show()


## Correlation (One Number Summary)

To summarize the relationship in a single statistic, we calculate the Pearson correlation between GDP per capita and the B-READY Public Services score. This does not imply causality; it only describes the strength and direction of association in this sample.


In [ ]:
corr = df["gdp_per_capita"].corr(df["public_services"])
print(f"Pearson correlation (GDP per capita vs Public Services): {corr:.3f}")

Pearson correlation (GDP per capita vs Public Services): 0.556


## Identifying Notable Outliers

To make the scatterplot more interpretable, we look for economies that perform much better or worse on Public Services than their income level would predict.

A simple way to do this is to compare each economy’s Public Services score to the average score within its approximate GDP range. Here, we take a simpler approach: we list the top and bottom performers on Public Services.


In [39]:
top_public_services = df.sort_values("public_services", ascending=False)[
    ["Economy", "gdp_per_capita", "public_services"]
].head(8)

bottom_public_services = df.sort_values("public_services", ascending=True)[
    ["Economy", "gdp_per_capita", "public_services"]
].head(8)

print("Top performers on Public Services:")
top_public_services


Top performers on Public Services:


,Economy,gdp_per_capita,public_services
13,Estonia,31428.354782,73.31
41,Singapore,90674.066633,70.40
11,Croatia,24050.439793,70.24
36,Portugal,29292.242105,69.53
19,Hungary,23292.326388,69.50
30,New Zealand,49205.179215,68.91
42,Slovak Republic,25992.674850,68.17
37,Rwanda,999.654562,67.37


## Interpretation

The scatterplot shows a moderate positive relationship between income levels and public services performance. This visual pattern is confirmed by the Pearson correlation coefficient of 0.556, indicating that higher-income economies tend, on average, to score better on public services for businesses.

This result aligns with expectations: wealthier economies often have more fiscal space, stronger administrative capacity, and greater digital infrastructure to support online public services such as business registration, taxation portals, and trade facilitation systems.

However, the relationship is far from deterministic. The data display substantial dispersion, especially among lower- and middle-income economies, suggesting that income alone does not fully explain public services quality.

Looking at the top performers on the Public Services pillar provides additional insight:
- Estonia and Singapore stand out as global leaders, combining high income levels with very strong public services scores. Their performance is consistent with their reputation for advanced digital government and streamlined business services.
- Portugal, Hungary, Croatia, New Zealand, and the Slovak Republic also cluster in the upper-right portion of the scatterplot, reinforcing the general income–performance relationship.

Importantly, the data also reveal meaningful exceptions:
- Rwanda appears among the top performers on Public Services despite having a GDP per capita under USD 1,000. This makes Rwanda a clear outlier and a particularly interesting case.

Rwanda’s strong score supports the B-READY report’s observation that high levels of public service adoption can be achieved even at lower income levels. This suggests that political commitment, institutional reform, and targeted digital investments can compensate, at least partially, for lower income.


## Conclusion

This analysis finds a moderate positive relationship between GDP per capita and business-facing public services, as measured by the B-READY Public Services pillar. The Pearson correlation of 0.556 indicates that wealthier economies generally provide stronger public services to businesses.

At the same time, the presence of notable outliers—particularly Rwanda—demonstrates that strong public services performance is not exclusive to high-income economies. This supports the B-READY report’s broader conclusion that digital public services can be advanced through institutional reforms and targeted investments, even in resource-constrained contexts.

Overall, the results suggest that while economic development facilitates public services quality, policy decisions and governance capacity play a crucial complementary role. Future analysis could build on this work by incorporating governance indicators or examining changes over time to better understand how economies improve public services performance beyond income effects alone.

